# Dependencies

In [1]:
!git clone https://github.com/microsoft/evodiff.git

Cloning into 'evodiff'...
remote: Enumerating objects: 2157, done.
remote: Counting objects: 100% (407/407), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 2157 (delta 371), reused 346 (delta 339), pack-reused 1750 (from 3)
Receiving objects: 100% (2157/2157), 20.22 MiB | 14.53 MiB/s, done.
Resolving deltas: 100% (1376/1376), done.
Updating files: 100% (159/159), done.


In [1]:
!pip install torch torchvision torchaudio
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.1.0+cpu.html
!pip install torch-geometric
!pip install evodiff
!pip install biotite
!pip install py3Dmol

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html


# Sequnce

In [1]:
pip install 'colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold'

  Cloning https://github.com/sokrypton/ColabFold to /tmp/pip-install-1refzqs9/colabfold_9f37ff2ed2bd448887d9ddb61ef55d1e
  Running command git clone --filter=blob:none --quiet https://github.com/sokrypton/ColabFold /tmp/pip-install-1refzqs9/colabfold_9f37ff2ed2bd448887d9ddb61ef55d1e
  Resolved https://github.com/sokrypton/ColabFold to commit 04545332cd2759528721711a86a2c8d28566a8c6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from google.colab import files
import os
import re
import hashlib
import random
import sys
import warnings
from Bio import BiopythonDeprecationWarning
from pathlib import Path
from colabfold.download import download_alphafold_params, default_data_dir
from colabfold.utils import setup_logging
from colabfold.batch import get_queries, run, set_model_type
from colabfold.plot import plot_msa_v2
import numpy as np
from colabfold.colabfold import plot_protein
from pathlib import Path
import matplotlib.pyplot as plt
import py3Dmol
import glob
from colabfold.colabfold import plot_plddt_legend, pymol_color_list, alphabet_list

def add_hash(x, y):
    return x + "_" + hashlib.sha1(y.encode()).hexdigest()[:5]

def run_colabfold_prediction(sequence, jobname='colabfold_job', template_mode='none', msa_mode='mmseqs2_uniref_env', use_amber=False):
    """
      Args:
        sequence (str): The protein sequence(s) to predict. Use ':' for multi-chain complexes.
        jobname (str): The name for the prediction job.
        template_mode (str): 'none', 'pdb100', or 'custom'.
        msa_mode (str): 'mmseqs2_uniref_env', 'mmseqs2_uniref', 'single_sequence', or 'custom'.
        use_amber (bool): Whether to relax the structure using Amber.
    """
    python_version = f"{sys.version_info.major}.{sys.version_info.minor}"


    jobname_base = re.sub(r'\W+', '', "".join(jobname.split()))
    jobname_hashed = add_hash(jobname_base, sequence)

    jobname_final = jobname_hashed
    n = 0
    while os.path.exists(jobname_final):
        jobname_final = f"{jobname_hashed}_{n}"
        n += 1

    os.makedirs(jobname_final, exist_ok=True)

    queries_path = os.path.join(jobname_final, f"{jobname_final}.csv")
    with open(queries_path, "w") as text_file:
        text_file.write(f"id,sequence\n{jobname_final},{sequence}")

    use_templates = (template_mode != "none")
    custom_template_path = os.path.join(jobname_final, "template") if template_mode == "custom" else None

    print("Jobname:", jobname_final)
    print("Sequence:", sequence)
    print("Length:", len(sequence.replace(":", "")))

    if not os.path.isfile("COLABFOLD_READY"):
        print("installing colabfold...")
        os.system("pip install -q --no-warn-conflicts 'colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold'")
        os.system(f"ln -s /usr/local/lib/python3.*/dist-packages/colabfold colabfold")
        os.system(f"ln -s /usr/local/lib/python3.*/dist-packages/alphafold alphafold")
        os.system("touch COLABFOLD_READY")
        if use_amber or use_templates:
          pass

    # --- MSA and Prediction setup ---
    a3m_file = os.path.join(jobname_final, f"{jobname_final}.a3m")
    if "mmseqs2" in msa_mode:
        pass
    elif msa_mode == "single_sequence":
        with open(a3m_file, "w") as text_file:
            text_file.write(f">1\n{sequence}")

    queries, is_complex = get_queries(queries_path)
    model_type = set_model_type(is_complex, "auto")
    download_alphafold_params(model_type, Path("."))

    # --- Run the prediction ---
    results = run(
        queries=queries,
        result_dir=jobname_final,
        use_templates=use_templates,
        custom_template_path=custom_template_path,
        num_relax=1 if use_amber else 0, # Simplified to 1 or 0
        msa_mode=msa_mode,
        model_type=model_type,
        num_models=5,
        num_recycles=3,
        relax_max_iterations=300,
        recycle_early_stop_tolerance=0.0,
        num_seeds=1,
        use_dropout=False,
        model_order=[1, 2, 3, 4, 5],
        is_complex=is_complex,
        data_dir=Path("."),
        pair_mode='unpaired_paired',
        pairing_strategy='greedy',
        save_all=False,
        save_recycles=False,
    )

    os.system(f"zip -r {jobname_final}.result.zip {jobname_final}")

    # --- Display 3D structure ---
    rank_num = 1
    color = "lDDT"
    show_sidechains = False
    show_mainchains = False

    tag = results["rank"][0][rank_num - 1]
    pdb_filename = f"{jobname_final}/{jobname_final}_unrelaxed_{tag}.pdb"
    pdb_file_list = glob.glob(pdb_filename)

    if not pdb_file_list:
        print("PDB file not found.")
        return

    model_name = f"rank_{rank_num}"
    view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
    view.addModel(open(pdb_file_list[0], 'r').read(), 'pdb')

    if color == "lDDT":
        view.setStyle({'cartoon': {'colorscheme': {'prop': 'b', 'gradient': 'roygb', 'min': 50, 'max': 90}}})
    elif color == "rainbow":
        view.setStyle({'cartoon': {'color': 'spectrum'}})

    if show_sidechains:
        BB = ['C', 'O', 'N']
        view.addStyle({'and': [{'resn': ["GLY", "PRO"], 'invert': True}, {'atom': BB, 'invert': True}]}, {'stick': {'colorscheme': f"WhiteCarbon", 'radius': 0.3}})
        view.addStyle({'and': [{'resn': "GLY"}, {'atom': 'CA'}]}, {'sphere': {'colorscheme': f"WhiteCarbon", 'radius': 0.3}})
        view.addStyle({'and': [{'resn': "PRO"}, {'atom': ['C', 'O'], 'invert': True}]}, {'stick': {'colorscheme': f"WhiteCarbon", 'radius': 0.3}})
    if show_mainchains:
        BB = ['C', 'O', 'N', 'CA']
        view.addStyle({'atom': BB}, {'stick': {'colorscheme': f"WhiteCarbon", 'radius': 0.3}})

    view.zoomTo()
    view.show()


# 1. Unconditional sequence generation




## 1.1 Generate a sequence with EvoDiff-Seq-OADM 38M





In [3]:
from evodiff.pretrained import OA_DM_38M
from evodiff.pretrained import OA_DM_640M

checkpoint = OA_DM_38M()
# checkpoint = OA_DM_640M()
model, collater, tokenizer, scheme = checkpoint

Downloading: "https://zenodo.org/record/8045076/files/oaar-38M.tar?download=1" to /root/.cache/torch/hub/checkpoints/oaar-38M.tar
100%|██████████| 434M/434M [09:01<00:00, 840kB/s] 


The only thing you need to define is the desired sequence length via `seq_len` input

In [4]:
from evodiff.generate import generate_oaardm

seq_len = 95
tokenized_sample, generated_sequence = generate_oaardm(model, tokenizer, seq_len, batch_size=1, device='cpu')
print("Generated sequence:", generated_sequence)
my_sequence = "".join(generated_sequence[0].split())

# Run the function
run_colabfold_prediction(my_sequence, jobname='prediction')

100%|██████████| 95/95 [00:13<00:00,  7.17it/s]


Generated sequence: ['MTMPPVHNCGFSALRQWAQEATSTQNWDCDLNAHGRDLERSCSIFVRVSGHKVKAVFAVGFGANGRYASIATFVASPNIPMSPTIGSMIASCIGI']
Jobname: prediction_8f125
Sequence: MTMPPVHNCGFSALRQWAQEATSTQNWDCDLNAHGRDLERSCSIFVRVSGHKVKAVFAVGFGANGRYASIATFVASPNIPMSPTIGSMIASCIGI
Length: 95
installing colabfold...


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:20 remaining: 00:00]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## 1.2 Generate a sequence with EvoDiff-D3PM-Uniform 38M


In [ ]:
from evodiff.pretrained import D3PM_UNIFORM_38M
from evodiff.pretrained import D3PM_UNIFORM_640M

checkpoint = D3PM_UNIFORM_38M(return_all=True)
# checkpoint = D3PM_UNIFORM_640M(return_all=True)
model, collater, tokenizer, scheme, timestep, Q_bar, Q = checkpoint

sohl-dickstein


Downloading: "https://zenodo.org/record/8045076/files/d3pm-uniform-38M.tar?download=1" to /root/.cache/torch/hub/checkpoints/d3pm-uniform-38M.tar
100%|██████████| 434M/434M [07:05<00:00, 1.07MB/s]


We can then generate 1 sequence via the following, where again only `seq_len` needs to be defined:

In [ ]:
from evodiff.generate import generate_d3pm

seq_len = 95

tokenized_sample, generated_sequence = generate_d3pm(model, tokenizer, Q, Q_bar, timestep, seq_len, batch_size=1, device='cpu')
# MRPIQSLLVIAAIMAFIAYIDLTFAFSRLMLLIHPFQVFDLMSILYILLYLVLIEKHKLRNRHIPTLKYIFISVGLLLPLIAAINAILILLNTTL
print(generated_sequence)
my_sequence = "".join(generated_sequence[0].split())

# Run the function
run_colabfold_prediction(my_sequence, jobname='prediction')

100%|██████████| 499/499 [01:05<00:00,  7.66it/s]


final seq ['NSQGQHYHLKMKEQWPNCSCDWSRFGIRRGKLTERLSVPRLLFANPSFVAGEQIYEAITWYTVPDEELKEEEHEETGLTISADERIPGDIQQIYL']
['NSQGQHYHLKMKEQWPNCSCDWSRFGIRRGKLTERLSVPRLLFANPSFVAGEQIYEAITWYTVPDEELKEEEHEETGLTISADERIPGDIQQIYL']
Jobname: prediction_f9392
Sequence: NSQGQHYHLKMKEQWPNCSCDWSRFGIRRGKLTERLSVPRLLFANPSFVAGEQIYEAITWYTVPDEELKEEEHEETGLTISADERIPGDIQQIYL
Length: 95


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:34 remaining: 00:00]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

# 2.  Conditional generation

To generate a sequence, given a multiple sequence alignment, you must have an MSA avaialble. Our `generate-msa.py` code samples the validation dataset of openfold, then subsamples an MSA `n_sequences` x `seq_length`, and generates a new query sequence for that sampled MSA.

To run the following code on a custom MSA, you must provide the path to an MSA saved as an A3M file and specify the subsampling of `n_sequences` by `seq_length` via the scheme of your choice (`selection_type=random` or `MaxHamming`), where the query sequence is the sequence you want to generate. We have not extensively tested our subsampling code outside of the Openfold dataset.


Note: All our conditional generation uses OADM models, currently we do not support conditional generation with D3PM

## 2.1 Evolutionary guided sequence generation with EvoDiff-MSA



In [ ]:
from evodiff.pretrained import MSA_OA_DM_MAXSUB

checkpoint = MSA_OA_DM_MAXSUB()
model, collater, tokenizer, scheme = checkpoint

Downloading: "https://zenodo.org/record/8045076/files/msa-oaar-maxsub.tar?download=1" to /root/.cache/torch/hub/checkpoints/msa-oaar-maxsub.tar
100%|██████████| 1.28G/1.28G [18:17<00:00, 1.26MB/s]


We have provided a random test example a3m file under `examples/a3m_example` which we will use here.

In [ ]:
from evodiff.generate_msa import generate_query_oadm_msa_simple
import re

path_to_msa = '/content/evodiff/examples/example_files/bfd_uniclust_hits.a3m'
n_sequences=64 # number of sequences in MSA to subsample
seq_length=256 # maximum sequence length to subsample
selection_type='random' # or 'MaxHamming'; MSA subsampling scheme


tokenized_sample, generated_sequence  = generate_query_oadm_msa_simple(path_to_msa, model, tokenizer, n_sequences, seq_length, device='cpu', selection_type=selection_type)


print("New sequence (no gaps, pad tokens)", re.sub('[!-]', '', generated_sequence[0][0],))
my_sequence_new = "".join(generated_sequence[0].split())

# Run the function
run_colabfold_prediction(my_sequence_new, jobname='prediction')

  0%|          | 0/164 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
  2%|▏         | 4/164 [03:54<2:35:35, 58.35s/it]

# 3. Inpainting and Scaffolding

## 3.1 Inpainting IDRs

#### Inpainting IDRs with EvoDiff-Seq

In [ ]:
from evodiff.pretrained import OA_DM_38M

checkpoint = OA_DM_38M()
model, collater, tokenizer, scheme = checkpoint

Using an exemplary input `sequence`, we will show you how to inpaint a new region (from `start_idx` to `end_idx`) of that sequence using EvoDiff-Seq

In [ ]:
from evodiff.conditional_generation import inpaint_simple

sequence = 'DQTERTVRSFEGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI'
start_idx = 20
end_idx = 50


sample, entire_sequence, generated_idr = inpaint_simple(model, sequence, start_idx, end_idx, tokenizer=tokenizer, device='cpu')

print("original sequence:", sequence)
print("generated sequence", entire_sequence)


print("\noriginal region:", sequence[start_idx:end_idx])
print("generated region:", generated_idr)
# ANGRQVHVAVVEHPWPIGMFWSSALVVARE
my_sequence_new = "".join(generated_idr[0].split())
my_sequence_old = "".join(sequence.split())
# Run the function
run_colabfold_prediction(my_sequence_old, jobname='prediction')
run_colabfold_prediction(my_sequence_new, jobname='prediction')

100%|██████████| 30/30 [00:06<00:00,  4.75it/s]


original sequence: DQTERTVRSFEGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI
generated sequence DQTERTVRSFEGRRTAPYLDISALAPNVHDCVTFYDFARMEFALTQAGPTPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI

original region: SRNVLTIGYGHLLNRPGANKSWEGRLTSAL
generated region: ISALAPNVHDCVTFYDFARMEFALTQAGPT
Jobname: prediction_423cd
Sequence: DQTERTVRSFEGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI
Length: 164


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:33 remaining: 00:00]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Jobname: prediction_ca73a
Sequence: I
Length: 1


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:00 remaining: 00:00]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

#### Inpainting IDRs with EvoDiff-MSA



Inpainting via EvoDiff-MSAs then follows the same approach as conditional-generation of a query sequence, given an MSA. One must ensure critical care in the subsampling of MSAs in this task, and ensure that the subsampled indices correspond to the correct regions you are interested in subsampling.

## 3.2 Scaffolding functional motifs

#### Scaffolding functional motifs with EvoDiff-Seq



Below we provide an example of scaffolding the PDB-ID: 1PRW.

EvoDiff-Seq will search for a PDB file, given the 3 letter PDB code and attempt to download it if it cannot find it in the local directory. Raw PDB files can be missing residues, or have extra residues so we want to express extreme caution when using this code, to ensure that you are scaffolding the correct indices.  

First, provide the start (`start_idx`) and ending (`end_idx`) indices for the motif you are interested in scaffolding (the end index is includsive). If there are multiple domains, make sure these are indexed in numerical order. In the example below; `start_idx=[51,15]`, `end_idx=[70,34]` is not acceptable. For multiple domains, we retain the original spacing between motifs, and extract the entire domain from `start_idx[0]` to `end_idx[-1]` and fill the non-motif regions with a `MASK` token.

This code indexes a data folder `scaffolding-pdbs`, ensure that folder is in the working path

In [ ]:
import random
import os
# Define the folder path
folder_path = '/content/evodiff/examples/scaffolding-pdbs/'
all_files = os.listdir(folder_path)

# Filter for files that end with '.pdb'
pdb_files = [f for f in all_files if f.endswith('.pdb')]
pdb_code_full = random.choice(pdb_files)
pdb_code = pdb_code_full.split('.')[0]

start_idx = [15, 51]
end_idx = [34, 70]

num_seqs = 1

data_top_dir = './' # Change this filepath to represent where scaffolding-msas and scaffolding-pdbs exists, this should be in the same folder as this notebook

In [ ]:
from evodiff.conditional_generation import generate_scaffold

scaffold_length = 75

generated_sequence, new_start_idx, new_end_idx = generate_scaffold(model, pdb_code, start_idx, end_idx, scaffold_length, data_top_dir, tokenizer, device='cpu')

print("motif start indices", new_start_idx)
print("motif end indices", new_end_idx)



DOWNLOADING PDB FILE FROM https://files.rcsb.org/download/1bcf_reference.pdb


HTTPError: HTTP Error 400: Bad Request

#### Scaffolding functional motifs with EvoDiff-MSA



EvoDiff-MSA requires an a3m formatted MSA in the data folder to proceed, for simplicity we did not wrap any homology tools for automatic MSA-generation. For generation, you must create an A3M, subsample an alignment (preserving the correct indices), and use this to generate a new query-sequence.

If you would like to analyze the generated structure by comparing it to the original using the RMSD score, look at the analysis/rmsd_analysis.py script

# 4. Scaffolding-pdbs Example

We provide PDB files in the ` examples/scaffolding-pdbs` folder. You can use the following code segment to visualize the various PDB files and pick one.

In [ ]:
!{sys.executable} -m pip install py3Dmol

/bin/bash: line 1: {sys.executable}: command not found


In [ ]:
import random
import py3Dmol
import os
# Define the folder path
folder_path = '/content/evodiff/examples/scaffolding-pdbs/'

# Get a list of all files in the folder
all_files = os.listdir(folder_path)

# Filter for files that end with '.pdb'
pdb_files = [f for f in all_files if f.endswith('.pdb')]

    # Randomly select one filename from the list
random_pdb_filename = random.choice(pdb_files)

    # The rest of your visualization code
view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
view.addModel(open(os.path.join(folder_path, random_pdb_filename), 'r').read(), 'pdb')
view.setStyle({'cartoon': {'colorscheme': {'prop':'b','gradient': 'roygb','min':0.2,'max':0.9}}})
view.setStyle({'cartoon': {'color':'spectrum'}})
view.zoomTo()
view.show()
print(f'\n---------------------------   {random_pdb_filename}   ---------------------------')

3Dmol.js failed to load for some reason. Please check your browser console for error messages.


---------------------------   7mrx_clean.pdb   ---------------------------


In [6]:
s = 'DQTERTVRSFEGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI'
s_list = list(s)
for i in range(3, len(s_list), 4):
    s_list[i] = 'F'
modified_s = "".join(s_list)
print(f"The original is:\n{s}\n")
print(f"The modifie is:\n{modified_s}")


The original is:
DQTERTVRSFEGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI

The modifie is:
DQTFRTVFSFEFRRTFPYLFSRNFLTIFYGHFLNRFGANFSWEFRLTFALPFEFKFRLTFLAAFQLHFTDVFLATFRAQFLYGFGAYFESVFVSLFDLWFDSVFNLGFRKLFNWSFLRTFLESFDWGFAAKFLGRFTFGFEPVFRRMFESMFMRRFIDLFHYNF


In [7]:
run_colabfold_prediction(s, jobname='prediction')

Jobname: prediction_423cd
Sequence: DQTERTVRSFEGRRTAPYLDSRNVLTIGYGHLLNRPGANKSWEGRLTSALPREFKQRLTELAASQLHETDVRLATARAQALYGSGAYFESVPVSLNDLWFDSVFNLGERKLLNWSGLRTKLESRDWGAAAKDLGRHTFGREPVSRRMAESMRMRRGIDLNHYNI
Length: 164


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:02 remaining: 00:00]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
run_colabfold_prediction(modified_s, jobname='prediction')

Jobname: prediction_1f47d
Sequence: DQTFRTVFSFEFRRTFPYLFSRNFLTIFYGHFLNRFGANFSWEFRLTFALPFEFKFRLTFLAAFQLHFTDVFLATFRAQFLYGFGAYFESVFVSLFDLWFDSVFNLGFRKLFNWSFLRTFLESFDWGFAAKFLGRFTFGFEPVFRRMFESMFMRRFIDLFHYNF
Length: 164


COMPLETE: 100%|██████████| 150/150 [elapsed: 00:16 remaining: 00:00]


3Dmol.js failed to load for some reason. Please check your browser console for error messages.